# V3
* Training, Validation Data 분리
* 학습 시켜봤는데 InternalError 이런 애러가 뜸
* 사과 이미지를 shape 해봤는데 이미지 하나당 픽셀이 (1000,1000) 이었음...
* 이미지 하나당 픽셀 수가 1,000,000개여서 gpu의 memory를 초과했다고함.
* 모든 이미지 픽셀 수 줄이기
* 만약 그래도 GPU의 Memory를 초과했다고 하면 SVD를 사용하여 원본 이미지를 최대한 해치지 않으면서 feature값을 최소화하려함
* 그래서 SVD를 사용하여 원본 이미지를 최대한 해치지 않으면서 픽셀 수를 줄인 이미지를 만들어 보려함

# GPU

In [1]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
session = tf1.Session(config=config)

2022-11-05 06:48:53.547846: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 06:48:54.154897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4053 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:b3:00.0, compute capability: 8.0


# Training Data

In [2]:
import os
import numpy as np
from PIL import Image




# Training_L
image_path = "Data/Training/L/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Tra_L = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Tra_L.append(img_array)
    
    
    
    
# Training_M
image_path = "Data/Training/M/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Tra_M = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Tra_M.append(img_array)
    
    
    
    
# Training_S
image_path = "Data/Training/S/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Tra_S = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Tra_S.append(img_array)
    

    
    
# Total Training Data
X_train = Tra_L + Tra_M + Tra_S


X_train_np = np.array(X_train)


y_train = []

for i in ["L", "M", "S"]:
    for j in range(2380):
        y_train.append(i)
        
y_train_np = np.array(y_train)

### train_test_split

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_np, y_train_np
                                       , test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5712, 1000, 1000, 3), (1428, 1000, 1000, 3), (5712,), (1428,))

# Validation
* train으로 만든 모델을 검증하기 위해 우리는 test data만 사용할 것입니다.
이때, 고정된 test data만 가지고 모델의 성능을 확인하고, 파라미터를 수정하고, 또 확인하고, 수정하고,... 이 과정을 반복하게 되면 결국 최종 모델을 test data에만 성능이 좋은 모델이 될 것입니다. 즉, test data에 과적합이 되어 다른 data가 들어왔을 때는 예측력이 떨어질 수 있습니다.

#### Train Data
- 모델을 학습하기 위한 데이터셋으로 이때 학습은 최적의 파라미터를 찾는 것입니다.

#### Validation Data
- 학습이 이미 완료된 모델을 검증하기 위한 데이터셋
- 학습이 된 여러 모델 중 가장 좋은 하나의 모델을 고르기 위한 데이터셋
- 학습 과정에 어느 정도 관여
- 하지만 Validation  데이터 자체가 학습에 직접적으로 관여하는 것은 아님

#### Test Data
- 모델의 최종 성능을 평가하기 위한 데이터셋

#### Cross Validation
- 필요하다면...

In [4]:
import os
import numpy as np
from PIL import Image




# Validation_L
image_path = "Data/Validation/L/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Val_L = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Val_L.append(img_array)
    


    
# Validation_M
image_path = "Data/Validation/M/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Val_M = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Val_M.append(img_array)
    
    
    
    
# Validation_S
image_path = "Data/Validation/S/"

img_list = os.listdir(image_path)
img_list_jpg = [img for img in img_list if img.endswith(".png")]

Val_S = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    Val_S.append(img_array)
    
    

    
# Total Validaiont Data
X_val = Val_L + Val_M + Val_S


X_val_np = np.array(X_val)


y_val = []

for i in ["L", "M", "S"]:
    for j in range(340):
        y_val.append(i)
        
y_val_np = np.array(y_val)

# BLAS 
* TensorFlow에서 두개의 일을 동시에 하면 생기는 에러 ---> restart
* 사용할 수 있는 메모리를 초과해서 사용하면 생기는 에러 ---> 차원 축소

In [15]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[1000, 1000]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(10, activation="softmax")])

model.summary()

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

history = model.fit(X_val_np, y_val_np, epochs=10,
                   validation_data=(X_val_np, y_val_np))

2022-11-05 10:47:25.130115: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 381.47MiB (rounded to 400000000)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-11-05 10:47:25.130230: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-11-05 10:47:25.130273: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 141, Chunks in use: 141. 35.2KiB allocated for chunks. 35.2KiB in use in bin. 5.1KiB client-requested in use in bin.
2022-11-05 10:47:25.130302: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 5, Chunks in use: 5. 3.8KiB allocated for chunks. 3.8KiB in use in bin. 2.6KiB client-requested in use in bin.
2022-11-05 10:47:25.130330: I tens

ResourceExhaustedError: failed to allocate memory [Op:AddV2]